# How to use nomad-utility-workflows to perform NOMAD API Calls

Imports for the following examples:

In [1]:
# ! If needed !
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import time
from pprint import pprint

from decouple import config as environ

from nomad_utility_workflows.utils.core import get_authentication_token
from nomad_utility_workflows.utils.datasets import (
    create_dataset,
    delete_dataset,
    get_dataset_by_id,
    retrieve_datasets,
)
from nomad_utility_workflows.utils.entries import (
    download_entry_by_id,
    get_entries_of_my_uploads,
    get_entries_of_upload,
    get_entry_by_id,
    query_entries,
)
from nomad_utility_workflows.utils.uploads import (
    delete_upload,
    edit_upload_metadata,
    get_all_my_uploads,
    get_upload_by_id,
    publish_upload,
    upload_files_to_nomad,
)
from nomad_utility_workflows.utils.users import (
    get_user_by_id,
    search_users_by_name,
    who_am_i,
)

## NOMAD URLs

The NOMAD URL specifies the base address of the API for the NOMAD deployment of interest. Typically, this URL is structured as `https://<deployment_base_path>/api/v1`.

By default, nomad-utility-workflows uses the Test deployment of NOMAD to make API calls. This is simply a safety mechanism so that users do not accidentally publish something during testing. 

All API functions allow the user to specify the URL with the optional keyword argument `url`. If you want to use the central NOMAD URLs, you can simply set `url` equal to "prod", "staging", or "test", which correspond to the following deployments (see full URLs below):

- prod: the official NOMAD deployment. 
    - Updated most infrequently (as advertised in #software-updates on the NOMAD Discord Server)
- staging: the beta version of NOMAD. 
    - Updated more frequently than prod in order to integrate and test new features. 
- test: a test NOMAD deployment. 
    - The data is occassionally wiped, such that test publishing can be made.

Note that the prod and staging deployments share a common database, and that publishing on either will result in publically available data.

Alternatively to these short names, the user can use the `url` input to specify the full API address to some alternative NOMAD deployment, e.g., an Oasis.

For reference, the full addresses of the above-mentioned central NOMAD deployments (including api suffix) are:

In [3]:
from nomad_utility_workflows.utils.core import (
    NOMAD_PROD_URL,
    NOMAD_STAGING_URL,
    NOMAD_TEST_URL,
)

print(NOMAD_PROD_URL, NOMAD_STAGING_URL, NOMAD_TEST_URL)

https://nomad-lab.eu/prod/v1/api/v1 https://nomad-lab.eu/prod/v1/staging/api/v1 https://nomad-lab.eu/prod/v1/test/api/v1


## Authentication

Some API calls, e.g., making uploads or accessing your own non-published uploads, require an authentication token. To generate this token, nomad-utility-workflows expects that your NOMAD credentials are stored in a `.env` file in the plugin root directory in the format:

```bash
NOMAD_USERNAME="<your_nomad_username>"
NOMAD_PASSWORD="<your_nomad_password>"
```

You can access these explicitly with:

In [4]:
NOMAD_USERNAME = environ('NOMAD_USERNAME')
NOMAD_PASSWORD = environ('NOMAD_PASSWORD')
NOMAD_USERNAME

'JFRudzinski'

Use `get_authentication_token()` with your credentials to explicitly obtain and store a token:



In [5]:
token = get_authentication_token(
    username=NOMAD_USERNAME, password=NOMAD_PASSWORD, url='test'
)
token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJmb1hmZnM5QlFQWHduLU54Yk5PYlExOFhnZnlKU1FNRkl6ZFVnWjhrZzdVIn0.eyJleHAiOjE3NDcwNTIxODcsImlhdCI6MTc0Njk2NTc4NywianRpIjoiZDM0ZjNlZTktMzM1Mi00ZjZlLTkzYWItMjNhYWIzNmE0ODdmIiwiaXNzIjoiaHR0cHM6Ly9ub21hZC1sYWIuZXUvZmFpcmRpL2tleWNsb2FrL2F1dGgvcmVhbG1zL2ZhaXJkaV9ub21hZF9wcm9kIiwic3ViIjoiOGYwNTJlMWYtMTkwNi00MWZkLWIyZWItNjkwYzAzNDA3Nzg4IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoibm9tYWRfcHVibGljIiwic2lkIjoiZjg1MGZkZGItMjhiMS00YTUyLWJlOGUtMjg3MzYwOWUxMzMxIiwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBlbWFpbCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiSm9zZXBoIFJ1ZHppbnNraSIsInByZWZlcnJlZF91c2VybmFtZSI6ImpmcnVkemluc2tpIiwic2Vzc2lvbl9zdGF0ZSI6ImY4NTBmZGRiLTI4YjEtNGE1Mi1iZThlLTI4NzM2MDllMTMzMSIsImdpdmVuX25hbWUiOiJKb3NlcGgiLCJmYW1pbHlfbmFtZSI6IlJ1ZHppbnNraSIsImVtYWlsIjoicnVkemluc2tpQG1waXAtbWFpbnoubXBnLmRlIn0.Hc-2llW2iEj7Vu03vkDIXnwcDJyn7rBvMn8pR8NBEvqPhbrg3eK9OmSmhH-uIYP6OaA9bkYlJxJmE1EewU3IZnMUhVtfgT7ZjaWyi9tYDDSx9GmRrveVKjIrqVzd6c4WBXp6U-bKFlHYVwDF9YDpPmP_Hn1_BWsL5I71oDMimFi

In practice, you do not need to obtain a token yourself when using nomad-utility-workflows. A token will automatically be obtained for API calls that require authentication. However, you may want to do the token generation yourself for custom API calls (see `Writing your own wrappers` below.)

### NOMAD User Metadata

nomad-utility-workflows uses the `NomadUser()` class to store the following user metadata:

```python
class NomadUser:
    user_id: str
    name: str
    first_name: str 
    last_name: str 
    username: str 
    affiliation: str 
    affiliation_address: str 
    email: str
    is_oasis_admin: bool 
    is_admin: bool
    repo_user_id: str 
    created: dt.datetime
```


You can retrieve your own personal info with the `who_am_i()` function:

In [6]:
nomad_user_me = who_am_i(url='test')
nomad_user_me

NomadUser(name='Joseph Rudzinski')

Similarly, you can query NOMAD for other users with `search_users_by_name()`:

In [7]:
nomad_users = search_users_by_name('Rudzinski', url='test')
nomad_users

[NomadUser(name='Joseph Rudzinski'), NomadUser(name='Joseph Rudzinski')]

In the case of multiple matches or for robustly identifying particular users, e.g., coauthors, in the future, it may be useful to store their `user_id`&mdash;a persistent identifier for each user account. Then, in the future you can use `get_user_by_id()` to grab the user info:

In [8]:
nomad_user = get_user_by_id(nomad_users[0].user_id, url='test')
nomad_user

NomadUser(name='Joseph Rudzinski')

### Uploading Data

nomad-utility-workflows uses the `NomadUpload()` class to store the following upload metadata:

```python
class NomadUpload:
    upload_id: str
    upload_create_time: dt.datetime
    main_author: NomadUser
    process_running: bool
    current_process: str
    process_status: str
    last_status_message: str
    errors: list[Any]
    warnings: list[Any]
    coauthors: list[str]
    coauthor_groups: list[Any]
    reviewers: list[NomadUser]
    reviewer_groups: list[Any]
    writers: list[NomadUser]
    writer_groups: list[Any]
    viewers: list[NomadUser]
    viewer_groups: list[Any]
    published: bool
    published_to: list[Any]
    with_embargo: bool
    embargo_length: float
    license: str
    entries: int
    n_entries: int
    upload_files_server_path: str
    publish_time: dt.datetime
    references: list[str] 
    datasets: list[str] 
    external_db: str 
    upload_name: str
    comment: str 
    url: str
    complete_time: dt.datetime
```

You can make an upload using the `upload_files_to_nomad()` function with input `filename=<path_to_a_zip_file_with_your_upload_data>`, as follows:  

In [9]:
test_upload_fnm = (
    './test.zip'  # a dummy upload file containing a single empty json file
)

In [10]:
upload_id = upload_files_to_nomad(filename=test_upload_fnm, url='test')
upload_id

'47fM2TYlT5mRbl-wbms9YA'

### Checking the upload status

The returned `upload_id` can then be used to directly access the upload, e.g., to check the upload status, using `get_upload_by_id()`:

In [11]:
nomad_upload = get_upload_by_id(upload_id, url='test')

pprint(nomad_upload)

NomadUpload(upload_id='47fM2TYlT5mRbl-wbms9YA',
            upload_create_time=datetime.datetime(2025, 5, 11, 12, 16, 47, 38000),
            main_author=NomadUser(name='Joseph Rudzinski'),
            process_running=False,
            current_process='process_upload',
            process_status='SUCCESS',
            last_status_message='Process process_upload completed successfully',
            errors=[],
            warnings=[],
            coauthors=[],
            coauthor_groups=[],
            reviewers=[],
            reviewer_groups=[],
            writers=[NomadUser(name='Joseph Rudzinski')],
            writer_groups=[],
            viewers=[NomadUser(name='Joseph Rudzinski')],
            viewer_groups=[],
            published=False,
            published_to=[],
            with_embargo=False,
            embargo_length=0.0,
            license='CC BY 4.0',
            entries=1,
            n_entries=None,
            upload_files_server_path='/nomad/test/fs/staging/4/4

One common usage of this function is to ensure that an upload has been processed successfully before making a subsequent action on it, e.g., editing the metadata or publishing. For this purpose, one could require the `process_running==False` or `process_status='SUCCESS'`, e.g.:

```python
    import time

    max_wait_time = 20 * 60  # 20 minutes in seconds
    interval = 2 * 60  # 2 minutes in seconds
    elapsed_time = 0

    while elapsed_time < max_wait_time:
        nomad_upload = get_upload_by_id(upload_id, url='test')
        
        # Check if the upload is complete
        if nomad_upload.process_status == 'SUCCESS':
            break
        
        # Wait for 2 minutes before the next call
        time.sleep(interval)
        elapsed_time += interval
    else:
        raise TimeoutError("Maximum wait time of 20 minutes exceeded. Upload is not complete.")
```

### Editing the upload metadata

After your upload is processed successfully, you can add coauthors, references, and other comments, as well as link to a dataset and provide a name for the upload. Note that the coauthor is specified by an email address that should correspond to the email linked to the person's NOMAD account, which can be accessed from `NomadUser.email`. The metadata should be stored as a dictionary as follows:

```python
metadata = {
    "metadata": {
    "upload_name": '<new_upload_name>',
    "references": ["https://doi.org/xx.xxxx/xxxxxx"],
    "datasets": '<dataset_id>',
    "embargo_length": 0,
    "coauthors": ["coauthor@affiliation.de"],
    "comment": 'This is a test upload...'
    },
}
```

For example:

In [12]:
metadata_new = {'upload_name': 'Test Upload', 'comment': 'This is a test upload...'}
edit_upload_metadata(upload_id, url='test', upload_metadata=metadata_new)

{'upload_id': '47fM2TYlT5mRbl-wbms9YA',
 'data': {'process_running': True,
  'current_process': 'edit_upload_metadata',
  'process_status': 'PENDING',
  'last_status_message': 'Pending: edit_upload_metadata',
  'errors': [],
  'warnings': [],
  'complete_time': '2025-05-11T12:16:48.319000',
  'upload_id': '47fM2TYlT5mRbl-wbms9YA',
  'upload_create_time': '2025-05-11T12:16:47.038000',
  'main_author': '8f052e1f-1906-41fd-b2eb-690c03407788',
  'coauthors': [],
  'coauthor_groups': [],
  'reviewers': [],
  'reviewer_groups': [],
  'writers': ['8f052e1f-1906-41fd-b2eb-690c03407788'],
  'writer_groups': [],
  'viewers': ['8f052e1f-1906-41fd-b2eb-690c03407788'],
  'viewer_groups': [],
  'published': False,
  'published_to': [],
  'with_embargo': False,
  'embargo_length': 0,
  'license': 'CC BY 4.0',
  'entries': 1,
  'upload_files_server_path': '/nomad/test/fs/staging/4/47fM2TYlT5mRbl-wbms9YA'}}

Before moving on, let's again check that this additional process is complete:

In [13]:
nomad_upload = get_upload_by_id(upload_id, url='test')

pprint(nomad_upload.process_status == 'SUCCESS')
pprint(nomad_upload.process_running is False)

True
True


### Accessing individual entries of an upload

During the upload process, NOMAD automatically identfies representative files that indicate the presence of data that can be parsed with the plugins included within a given deployment. This means that each upload can contain multiple *entries*&mdash;the fundamental unit storage within the NOMAD database.

You can query the individual entries within a known upload with `get_entries_of_upload()`, which then returns the metadata within the `NomadEntry()` class of nomad-utility-worklfows:

```python
class NomadEntry:
    entry_id: str
    upload_id: str
    references: list[str]
    origin: str
    quantities: list[str] 
    datasets: list[NomadDataset] 
    n_quantities: int
    nomad_version: str
    upload_create_time: dt.datetime
    nomad_commit: str
    section_defs: list[NomadSectionDefinition] 
    processing_errors: list[Any]
    results: dict
    entry_name: str
    last_processing_time: dt.datetime
    parser_name: str
    calc_id: str
    published: bool
    writers: list[NomadUser]
    sections: list[str] 
    processed: bool
    mainfile: str
    main_author: NomadUser
    viewers: list[NomadUser] 
    entry_create_time: dt.datetime
    with_embargo: bool
    files: list[str] 
    entry_type: str
    authors: list[NomadUser] 
    license: str
    domain: str
    optimade: dict
    comment: str
    upload_name: str
    viewer_groups: list[Any]
    writer_groups: list[Any]
    text_search_contents: list[str]
    publish_time: dt.datetime 
    entry_references: list[dict]
    url: str
```

Let's try this out with our test upload. In this case, the upload is *not* published and located in the *private* `Your Uploads` section of the NOMAD deployment. To access the uploads there, we need to set `with_authentication=True`:

In [14]:
entries = get_entries_of_upload(upload_id, url='test', with_authentication=True)
pprint(f'Entries within upload_id={upload_id}:')
for entry in entries:
    pprint(f'entry_id={entry.entry_id}')

'Entries within upload_id=47fM2TYlT5mRbl-wbms9YA:'
'entry_id=ehktf2IROtsCfZAP5yR-tTo7RndQ'


To query an entry directly using the `entry_id`, use `get_entry_by_id()`:

In [15]:
entry = get_entry_by_id(entries[0].entry_id, url='test', with_authentication=True)
entry

NomadEntry(entry_id='ehktf2IROtsCfZAP5yR-tTo7RndQ', upload_id='47fM2TYlT5mRbl-wbms9YA', references=[], origin='Joseph Rudzinski', n_quantities=0, nomad_version='1.3.16rc2', upload_create_time=datetime.datetime(2025, 5, 11, 12, 16, 47, 38000, tzinfo=datetime.timezone.utc), nomad_commit='', processing_errors=[], entry_name='test.archive.json', last_processing_time=datetime.datetime(2025, 5, 11, 12, 16, 48, 5000, tzinfo=datetime.timezone.utc), parser_name='parsers/archive', calc_id='ehktf2IROtsCfZAP5yR-tTo7RndQ', published=False, writers=[NomadUser(name='Joseph Rudzinski')], processed=True, mainfile='test.archive.json', main_author=NomadUser(name='Joseph Rudzinski'), entry_create_time=datetime.datetime(2025, 5, 11, 12, 16, 47, 530000, tzinfo=datetime.timezone.utc), with_embargo=False, entry_type=None, license='CC BY 4.0', domain=None, comment='This is a test upload...', upload_name='Test Upload', text_search_contents=[], publish_time=None, entry_references=None, url='https://nomad-lab.eu/

You can download the full (meta)data stored in an entry using `download_entry_by_id()`. This will return the entire archive as a dictionary. If you supply a `zip_file_name` (including the desired local path), the raw data of the entry will also be downloaded and saved to a zip file. Otherwise, only the archive will be downloaded. 

In [16]:
test = download_entry_by_id(
    entry.entry_id,
    url='test',
    zip_file_name='./raw_entry_data.zip',
    with_authentication=True,
)
test

{'processing_logs': [{'event': 'Executing celery task',
   'proc': 'Entry',
   'process': 'process_entry',
   'process_worker_id': 'deRNBfBUQ1m9IkP7v4WtIg',
   'parser': 'parsers/archive',
   'logger': 'nomad.processing',
   'timestamp': '2025-05-11 12:16.47',
   'level': 'DEBUG'},
  {'exec_time': '0.0012881755828857422',
   'input_size': '3',
   'event': 'parser executed',
   'proc': 'Entry',
   'process': 'process_entry',
   'process_worker_id': 'deRNBfBUQ1m9IkP7v4WtIg',
   'parser': 'parsers/archive',
   'step': 'parsers/archive',
   'logger': 'nomad.processing',
   'timestamp': '2025-05-11 12:16.48',
   'level': 'INFO'},
  {'normalizer': 'MetainfoNormalizer',
   'step': 'MetainfoNormalizer',
   'event': 'normalizer completed successfully',
   'proc': 'Entry',
   'process': 'process_entry',
   'process_worker_id': 'deRNBfBUQ1m9IkP7v4WtIg',
   'parser': 'parsers/archive',
   'logger': 'nomad.processing',
   'timestamp': '2025-05-11 12:16.48',
   'level': 'INFO'},
  {'exec_time': '0.0

## Publishing Uploads

Once the processing of your upload is successful and you have added/adjusted the appropriate metadata, you can publish your upload with `publish_upload()`, making it publicly available on the corresponding NOMAD deployment. 

Note that once the upload is published you will no longer be able to make changes to the raw files that you uploaded. However, the upload metadata (accessed and edited in the above example) can be changed after publishing.

In [17]:
published_upload = publish_upload(nomad_upload.upload_id, url='test')
published_upload

{'upload_id': '47fM2TYlT5mRbl-wbms9YA',
 'data': {'process_running': True,
  'current_process': 'publish_upload',
  'process_status': 'PENDING',
  'last_status_message': 'Pending: publish_upload',
  'errors': [],
  'warnings': [],
  'complete_time': '2025-05-11T12:16:53.166000',
  'upload_id': '47fM2TYlT5mRbl-wbms9YA',
  'upload_name': 'Test Upload',
  'upload_create_time': '2025-05-11T12:16:47.038000',
  'main_author': '8f052e1f-1906-41fd-b2eb-690c03407788',
  'coauthors': [],
  'coauthor_groups': [],
  'reviewers': [],
  'reviewer_groups': [],
  'writers': ['8f052e1f-1906-41fd-b2eb-690c03407788'],
  'writer_groups': [],
  'viewers': ['8f052e1f-1906-41fd-b2eb-690c03407788'],
  'viewer_groups': [],
  'published': False,
  'published_to': [],
  'with_embargo': False,
  'embargo_length': 0,
  'license': 'CC BY 4.0',
  'entries': 1,
  'upload_files_server_path': '/nomad/test/fs/staging/4/47fM2TYlT5mRbl-wbms9YA'}}

## Finding and Creating Datasets

Although uploads can group multiple entries together, they are limited by the maximum upload size and act more as a practical tool for optimizing the transfer of data to the NOMAD repository. For scientifically relevant connections between entries, NOMAD uses *Datasets* and *Workflows*. 

You can easily create a dataset with `create_dataset()`:


In [23]:
import random

rand = random.randint(1, 1000000)
dataset_id = create_dataset(f'test dataset-{rand}', url='test')
dataset_id

'0iW24lycTFuQhsYnrF7yjw'

The returned `dataset_id` can then be used to add individual entries (or all entries within an upload) to the dataset by including it in the upload/entry metadata, using the method described above:

In [24]:
metadata_new = {'dataset_id': dataset_id}
edit_upload_metadata(upload_id, url='test', upload_metadata=metadata_new)

{'upload_id': '47fM2TYlT5mRbl-wbms9YA',
 'data': {'process_running': True,
  'current_process': 'edit_upload_metadata',
  'process_status': 'PENDING',
  'last_status_message': 'Pending: edit_upload_metadata',
  'errors': [],
  'warnings': [],
  'complete_time': '2025-05-11T12:17:04.976000',
  'upload_id': '47fM2TYlT5mRbl-wbms9YA',
  'upload_name': 'Test Upload',
  'upload_create_time': '2025-05-11T12:16:47.038000',
  'main_author': '8f052e1f-1906-41fd-b2eb-690c03407788',
  'coauthors': [],
  'coauthor_groups': [],
  'reviewers': [],
  'reviewer_groups': [],
  'writers': ['8f052e1f-1906-41fd-b2eb-690c03407788'],
  'writer_groups': [],
  'viewers': ['8f052e1f-1906-41fd-b2eb-690c03407788'],
  'viewer_groups': [],
  'published': True,
  'published_to': [],
  'publish_time': '2025-05-11T12:17:04.965000',
  'with_embargo': False,
  'embargo_length': 0,
  'license': 'CC BY 4.0',
  'entries': 1}}

In [25]:
nomad_upload = get_upload_by_id(upload_id, url='test')

pprint(nomad_upload.process_status == 'SUCCESS')
pprint(nomad_upload.process_running is False)

True
True




You can also retrieve the dataset metadata using the `dataset_id` with `get_dataset_by_id()`. The returned `NomadDataset()` class contains the following attributes:

```python
class NomadDataset:
    dataset_id: str
    dataset_create_time: dt.datetime
    dataset_name: str
    dataset_type: str
    dataset_modified_time: dt.datetime
    user: NomadUser
    doi: str
    pid: int
    m_annotations: dict
```

In [26]:
nomad_dataset = get_dataset_by_id(dataset_id, url='test')
nomad_dataset

NomadDataset(dataset_id='0iW24lycTFuQhsYnrF7yjw', dataset_create_time=datetime.datetime(2025, 5, 11, 12, 19, 1, 245000), dataset_name='test dataset-40425', dataset_type='owned', dataset_modified_time=datetime.datetime(2025, 5, 11, 12, 19, 1, 245000), user=NomadUser(name='Joseph Rudzinski'), doi=None, pid=None, m_annotations=None)

Alternatively, you can search for datasets, e.g., by `user_id` or `dataset_name`, using `retrieve_datasets()`:

In [27]:
my_datasets = retrieve_datasets(
    dataset_params={'user_id': nomad_user_me.user_id, 'max_datasets': 20}, url='test'
)
pprint(my_datasets)

[NomadDataset(dataset_id='EfhadCxpRaGpw50rWzK22w',
              dataset_create_time=datetime.datetime(2024, 10, 16, 10, 15, 5, 240000),
              dataset_name='test dataset',
              dataset_type='owned',
              dataset_modified_time=datetime.datetime(2024, 10, 16, 10, 15, 5, 240000),
              user=NomadUser(name='Joseph Rudzinski'),
              doi=None,
              pid=None,
              m_annotations=None),
 NomadDataset(dataset_id='i31BPDbER3iB3IJPS3_Iog',
              dataset_create_time=datetime.datetime(2025, 2, 26, 9, 23, 47, 831000),
              dataset_name='Example Dataset - DPG Tutorial 2025 - Rudzinski',
              dataset_type='owned',
              dataset_modified_time=datetime.datetime(2025, 2, 26, 9, 23, 47, 831000),
              user=NomadUser(name='Joseph Rudzinski'),
              doi=None,
              pid=None,
              m_annotations=None),
 NomadDataset(dataset_id='hgOLOSPJT4CQah0xoAGkEg',
              dataset_create_tim

To get the list of entries contained within a dataset, use `query_entries()`:

In [28]:
dataset_entries = query_entries(query_params={'dataset_id': dataset_id}, url='test')
for entry in dataset_entries:
    pprint(f'entry_id={entry.entry_id}, upload_id={entry.upload_id}')

'entry_id=ehktf2IROtsCfZAP5yR-tTo7RndQ, upload_id=47fM2TYlT5mRbl-wbms9YA'


There is no "publishing" action for datasets. Instead, when the dataset is complete (i.e., you are ready to lock the contents of the dataset), you can *assign a DOI*. There is currently no API action for this within nomad-utility-workflows. You must go to the GUI of the relevant deployment, go to `PUBLISH > Datasets`, find the dataset, and then click the "assign a DOI" banner icon to the right of the dataset entry.

## Deleting Uploads and Datasets

You can delete uploads and datasets using `delete_upload()` and `delete_dataset()` as demonstrated in the following examples (along with the previously explained workflow of uploading, editing, etc.). Note that the wait times in these examples are arbitrary. One should optimize these for specific use cases.

**upload, check for success, delete, check for success**:

In [29]:
# Make a dummy upload
upload_id = upload_files_to_nomad(filename=test_upload_fnm, url='test')


max_wait_time = 15  # 15 seconds
interval = 5  # 5 seconds
elapsed_time = 0

while elapsed_time < max_wait_time:
    # Get the upload
    nomad_upload = get_upload_by_id(upload_id, url='test')

    # Check if the upload is complete
    if nomad_upload.process_status == 'SUCCESS':
        break

    # Wait for 5 seconds before the next call
    time.sleep(interval)
    elapsed_time += interval
else:
    raise TimeoutError(
        'Maximum wait time of 15 seconds exceeded. Upload is not complete.'
    )

# Delete the upload
delete_upload(upload_id, url='test')

# Wait for 5 seconds to make sure deletion is complete
time.sleep(5)

# Check if the upload was deleted
try:
    get_upload_by_id(upload_id, url='test')
except Exception:
    pprint(f'Upload with upload_id={upload_id} was deleted successfully.')

'Upload with upload_id=mRlxToYvSG2d-BuWxTH4Mw was deleted successfully.'


**create dataset, check for success, delete, check for success**:

In [30]:
# Make a dummy dataset
dataset_id = create_dataset('dummy dataset', url='test')

# Wait for 5 seconds to make sure dataset is created
time.sleep(5)

# Ensure the dataset was created
dummy_dataset = get_dataset_by_id(dataset_id, url='test')
assert dummy_dataset.dataset_id == dataset_id

# Delete the upload
delete_dataset(dataset_id, url='test')

# Wait for 5 seconds to make sure deletion is complete
time.sleep(5)

# Check if the dataset was deleted
try:
    get_dataset_by_id(dataset_id, url='test')
except Exception:
    pprint(f'Dataset with dataset_id={dataset_id} was deleted successfully.')

'Dataset with dataset_id=7xUggwulThaSKa52nCH2Wg was deleted successfully.'


## Useful Wrappers

nomad-utility-workflows contains a few useful wrapper functions to help users query all of their uploads and corresponding entries:

In [31]:
get_all_my_uploads(url='test')

[NomadUpload(upload_id='bQa5SGDQQ8auQUBb5AaYHw', upload_create_time=datetime.datetime(2024, 10, 14, 10, 48, 40, 994000), main_author=NomadUser(name='Joseph Rudzinski'), process_running=False, current_process='publish_upload', process_status='SUCCESS', last_status_message='Process publish_upload completed successfully', errors=[], warnings=[], coauthors=[], coauthor_groups=[], reviewers=[], reviewer_groups=[], writers=[NomadUser(name='Joseph Rudzinski')], writer_groups=[], viewers=[NomadUser(name='Joseph Rudzinski')], viewer_groups=[], published=True, published_to=[], with_embargo=False, embargo_length=0.0, license='CC BY 4.0', entries=1, n_entries=None, upload_files_server_path=None, publish_time=datetime.datetime(2024, 10, 14, 10, 48, 55, 806000), references=None, datasets=None, external_db=None, upload_name='Test Upload', comment=None, url='https://nomad-lab.eu/prod/v1/test/api/v1', complete_time=datetime.datetime(2024, 10, 14, 10, 48, 55, 818000)),
 NomadUpload(upload_id='DN61X4r7SC

In [32]:
get_entries_of_my_uploads(url='test')

[NomadEntry(entry_id='ycdeXhPDG-nIgEQlqBfzIEKPWCvy', upload_id='bQa5SGDQQ8auQUBb5AaYHw', references=[], origin='Joseph Rudzinski', n_quantities=34, nomad_version='1.3.7.dev55+ge83de27b3', upload_create_time=datetime.datetime(2024, 10, 14, 10, 48, 40, 994000, tzinfo=datetime.timezone.utc), nomad_commit='', processing_errors=[], entry_name='test.archive.json', last_processing_time=datetime.datetime(2024, 10, 14, 10, 48, 42, 415000, tzinfo=datetime.timezone.utc), parser_name='parsers/archive', calc_id='ycdeXhPDG-nIgEQlqBfzIEKPWCvy', published=True, writers=[NomadUser(name='Joseph Rudzinski')], processed=True, mainfile='test.archive.json', main_author=NomadUser(name='Joseph Rudzinski'), entry_create_time=datetime.datetime(2024, 10, 14, 10, 48, 41, 672000, tzinfo=datetime.timezone.utc), with_embargo=False, entry_type=None, license='CC BY 4.0', domain=None, comment='This is a test upload...', upload_name='Test Upload', text_search_contents=[], publish_time=None, entry_references=None, url='h

### Writing Your Own Wrappers

In `nomad_utility_workflows.utils.core` you will find the core NOMAD API functions `get_nomad_request()`, `post_nomad_request()`, and `delete_nomad_request()`. Using these as a basis, along with the [NOMAD API Dashboard](https://nomad-lab.eu/prod/v1/staging/api/v1/extensions/docs#/), you can easily extend the `nomad-utility-workflows` module for making more specific queries within your specialized workflows.
